# Build a Corrective RAG with Strand Agents

In this notebook, we'll construct a corrective Retrieval-Augmented Generation (RAG) system using Strand Agents. We'll define new tools, integrate external APIs, and orchestrate them into an intelligent agent workflow.

<div style="text-align:left">
    <img src="images/architecture.png" width="100%" />
</div>

## Setup and Prerequisites

### Requirements

Before running this notebook, ensure the following prerequisites are completed:

- Run the `prerequisites` notebook in this folder
- Python 3.10 or later
- An AWS account
- Anthropic Claude 3.7 enabled in Amazon Bedrock
- IAM role with permissions for:
  - Amazon Bedrock Knowledge Base
  - Amazon S3
  - Amazon OpenSearch Serverless

We’ll start by installing all required packages.

In [ ]:
%pip install -qr requirements.txt

In [ ]:
import asyncio, re
from strands import Agent, tool
from langchain.schema import Document
from strands_tools import agent_graph, retrieve
from langchain_aws import ChatBedrockConverse, BedrockEmbeddings
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [ ]:
eval_modelId = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
thinking_params= {
    "thinking": {
        "type": "disabled"
    }
}
llm_for_evaluation = ChatBedrockConverse(model_id=eval_modelId, additional_model_request_fields=thinking_params)
llm_for_evaluation = LangchainLLMWrapper(llm_for_evaluation)
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
bedrock_embeddings = LangchainEmbeddingsWrapper(bedrock_embeddings)

### Set Up Tavily Web Search API

We will use the [Tavily](https://tavily.com/) API for external web searches when the knowledge base content is insufficient. You’ll be prompted to securely input your API key.


In [ ]:
import getpass
import os

def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")
        
_set_env("TAVILY_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

## Define Tools for the Agent

Next, we define the tools the agent will use. These include:

- A relevance evaluator that scores the retrieved chunks using RAGAs
- A web search fallback to enhance answers when KB content is not relevant


In [ ]:
@tool
def check_chunks_relevance(results: str, question: str):
    """
    Evaluates the relevance of retrieved chunks to the user question using RAGAs.

    Args:
        results (str): Retrieval output as a string with 'Score:' and 'Content:' patterns.
        question (str): Original user question.

    Returns:
        dict: A binary score ('yes' or 'no') and the numeric relevance score, or an error message.
    """
    try:
        if not results or not isinstance(results, str):
            raise ValueError("Invalid input: 'results' must be a non-empty string.")
        if not question or not isinstance(question, str):
            raise ValueError("Invalid input: 'question' must be a non-empty string.")

        # Extract content chunks using regex
        pattern = r"Score:.*?\nContent:\s*(.*?)(?=Score:|\Z)"
        docs = [chunk.strip() for chunk in re.findall(pattern, results, re.DOTALL)]

        if not docs:
            raise ValueError("No valid content chunks found in 'results'.")

        # Prepare evaluation sample
        sample = SingleTurnSample(
            user_input=question,
            response="placeholder-response",  # required dummy response
            retrieved_contexts=docs
        )

        # Evaluate using context precision metric
        scorer = LLMContextPrecisionWithoutReference(llm=llm_for_evaluation)
        score = asyncio.run(scorer.single_turn_ascore(sample))

        print("------------------------")
        print("Context evaluation")
        print("------------------------")
        print(f"chunk_relevance_score: {score}")

        return {
            "chunk_relevance_score": "yes" if score > 0.5 else "no",
            "chunk_relevance_value": score
        }

    except Exception as e:
        return {
            "error": str(e),
            "chunk_relevance_score": "unknown",
            "chunk_relevance_value": None
        }

In [ ]:
@tool
def web_search(query):
    """
    Perform web search based on the query and return results as Documents.
    Only to be used if chunk_relevance_score is no.

    Args:
        query (str): The user question or rephrased query.

    Returns:
        dict: {
            "documents": [Document, ...]  # list of Document objects with web results
        }
    """

    print("---WEB SEARCH---")

    # Perform web search
    docs = web_search_tool.invoke({"query": query})

    # Convert each result into a Document object
    documents = [Document(page_content=d["content"]) for d in docs]

    return {
        "documents": documents
    }


## Create the Agent

We initialize the agent with access to our tools and provide it with a system prompt to define its purpose and reasoning logic.

### Environment Configuration

Set the Knowledge Base ID, AWS region, and relevance threshold score as environment variables for the **retrive** tool to fetch.

In [ ]:
%store -r kb_id
%store -r kb_region

In [ ]:
os.environ["KNOWLEDGE_BASE_ID"] = kb_id #Change if you are using a different KB than the created in the prerequisites notebook
os.environ["AWS_REGION"] = kb_region #Change if needed
os.environ["MIN_SCORE"] = "0.2"

### Instantiate a simple RAG Agent
Let's create an agent which only uses the retrieve tool so we can compare it's results with the corrective RAG agent.

In [ ]:
simple_agent = Agent(
    tools=[retrieve],
    system_prompt="You are a rag agent. When a user asks you a question you will check it in your knowledge base. When you use the retrieve tool, do not modify the users question, pass as is.",
)

In [ ]:
result = simple_agent("Which is my medical insurance provider name and Allstate Insurance Company's phone number?")

You will notice the agent is not able to find the phone number in it's knowledge base. 

### Instantiate the corrective RAG Agent

We define the agent’s behavior using a system prompt and register the custom tools. The agent follows this reasoning flow:

1. Try to answer using the knowledge base
2. Evaluate chunk relevance using RAGAS
3. If content is not relevant, trigger web search


In [ ]:
corrective_agent = Agent(
    tools=[web_search, retrieve, check_chunks_relevance],
    system_prompt="You are a corrective rag agent. When a user asks you a question you will first check it in your knowledge base (if you can't answer it from the current conversation memory). You will evaluate if the returned chunks are relevant using a relevance score tool. If they are not relevant to the question you will use your web search tool to gather additional data to answer the question. You are an agent in charge of looking for information in your knoweldege base and if the results are not relevant using ragas, use a web search. When you use the retrieve tool, do not modify or break down the users question, pass as is.",
)

## Query the Agent

Let’s ask the agent a question that may require both internal and external data sources. It will:

1. Search the knowledge base
2. Score the relevance of results
3. Supplement with web search if needed
4. Return a complete answer


In [ ]:
result = corrective_agent("Which is my medical insurance provider name and Allstate Insurance Company's phone number?")

### View Execution Summary

Retrieve evaluation metrics from the agent's response for transparency and performance monitoring.


In [ ]:
result.metrics.get_summary()

## Clean up the resources

When you have finished with this notebook, return to the previous notebook to delete the Knowledge Base created and not incurr extra costs!